In [1]:
#!pip install geopandas
#!pip install imdlib
#!pip install rasterio

In [16]:
import geopandas as gpd
import imdlib as imd
import xarray as xr
import numpy as np
import pandas as pd

In [3]:
file_name = r"C:\Users\Kanishk Goyal\OneDrive - IIT Kanpur\Desktop\Prof. Hamim Zafar\Data\Shape file for Districts India\output.shp"
gdf = gpd.read_file(file_name)

In [4]:
statename = 'Uttar Pradesh'
my = gdf[gdf.statename == statename]

In [12]:
dist_list = pd.Series(my['distname'])
pd.set_option('display.max_columns', None)
dist_list_values = dist_list.tolist()

In [17]:
start_yr = 1995
end_yr = 2022

variable_tmin = 'tmin' # other options are ('tmin'/ 'tmax')
variable_tmax = 'tmax'
variable_rain = 'rain'

data_tmin = imd.get_data(variable_tmin, start_yr, end_yr, fn_format='yearwise')
data_tmax = imd.get_data(variable_tmax, start_yr, end_yr, fn_format='yearwise')
data_rain = imd.get_data(variable_rain, start_yr, end_yr, fn_format='yearwise')
ds_tmin = data_tmin.get_xarray() # Converting to xarray is also required for SLICING, for a date.
ds_tmax = data_tmax.get_xarray() # Converting to xarray is also required for SLICING, for a date.
ds_rain = data_rain.get_xarray() # Converting to xarray is also required for SLICING, for a date.

Downloading: mintemp for year 1995
Downloading: mintemp for year 1996
Downloading: mintemp for year 1997
Downloading: mintemp for year 1998
Downloading: mintemp for year 1999
Downloading: mintemp for year 2000
Downloading: mintemp for year 2001
Downloading: mintemp for year 2002
Downloading: mintemp for year 2003
Downloading: mintemp for year 2004
Downloading: mintemp for year 2005
Downloading: mintemp for year 2006
Downloading: mintemp for year 2007
Downloading: mintemp for year 2008
Downloading: mintemp for year 2009
Downloading: mintemp for year 2010
Downloading: mintemp for year 2011
Downloading: mintemp for year 2012
Downloading: mintemp for year 2013
Downloading: mintemp for year 2014
Downloading: mintemp for year 2015
Downloading: mintemp for year 2016
Downloading: mintemp for year 2017
Downloading: mintemp for year 2018
Downloading: mintemp for year 2019
Downloading: mintemp for year 2020
Downloading: mintemp for year 2021
Downloading: mintemp for year 2022
Download Successful 

In [18]:
def mask_avg(xarray_,district_shape):
    import rasterio
    import rasterio.features
    import numpy as np
    lon_values = xarray_.lon.values
    lat_values = xarray_.lat.values
    lon_spacing = lon_values[1] - lon_values[0]
    lat_spacing = lat_values[1] - lat_values[0]
    transform = rasterio.transform.from_origin(lon_values[0], lat_values[-1], lon_spacing, lat_spacing)

    mask = rasterio.features.geometry_mask(district_shape, out_shape=(xarray_.lat.size, xarray_.lon.size),
                                       transform=transform, invert=True)

    masked_xarray = xarray_.where(mask)

    for var_name in masked_xarray.data_vars:
        alpha = np.nanmean(masked_xarray[var_name])
    
    return alpha

In [19]:
def avg_value(year,district_shape):

    list_march_rain = []
    list_april_rain = []
    list_march_tmin = []
    list_april_tmin = []
    list_march_tmax = []
    list_april_tmax = []

    #print(1)

    for i in range(31):
        time_str = f'{year}-03-{i+1:02d}'
        d_tmin_march = ds_tmin.sel(time=time_str)
        d_tmax_march = ds_tmax.sel(time=time_str)
        d_rain_march = ds_rain.sel(time=time_str)
        list_march_tmin.append(d_tmin_march)
        list_march_tmax.append(d_tmax_march)
        list_march_rain.append(d_rain_march)
    
    for i, ds in enumerate(list_march_tmin):
        list_march_tmin[i] = ds.where(ds < 99, np.nan)
    for i, ds in enumerate(list_march_tmax):
        list_march_tmax[i] = ds.where(ds < 99, np.nan)  
    for i, ds in enumerate(list_march_tmin):
        list_march_rain[i] = ds.where(ds != -999, np.nan) 

    march_tmin = xr.concat(list_march_tmin, dim='dataset').mean(dim='dataset')
    march_tmax = xr.concat(list_march_tmax, dim='dataset').mean(dim='dataset')
    march_rain = xr.concat(list_march_rain, dim='dataset').mean(dim='dataset')
    
    tmin_march = mask_avg(march_tmin,district_shape)
    tmax_march = mask_avg(march_tmax,district_shape)
    rain_march = mask_avg(march_rain,district_shape)

    for j in range(30):
        time_str = f'{year}-04-{j+1:02d}'
        d_tmin_april = ds_tmin.sel(time=time_str)
        d_tmax_april = ds_tmax.sel(time=time_str)
        d_rain_april = ds_rain.sel(time=time_str)
        list_april_tmin.append(d_tmin_april)
        list_april_tmax.append(d_tmax_april)
        list_april_rain.append(d_rain_april)
    
    for i, ds in enumerate(list_april_tmin):
        list_april_tmin[i] = ds.where(ds < 99, np.nan)
    for i, ds in enumerate(list_april_tmax):
        list_april_tmax[i] = ds.where(ds < 99, np.nan)  
    for i, ds in enumerate(list_april_tmin):
        list_april_rain[i] = ds.where(ds != -999, np.nan) 

    april_tmin = xr.concat(list_april_tmin, dim='dataset').mean(dim='dataset')
    april_tmax = xr.concat(list_april_tmax, dim='dataset').mean(dim='dataset')
    april_rain = xr.concat(list_april_rain, dim='dataset').mean(dim='dataset')
    
    tmin_april = mask_avg(april_tmin,district_shape)
    tmax_april = mask_avg(april_tmax,district_shape)
    rain_april = mask_avg(april_rain,district_shape)

    #print(2)



    return tmin_march,tmax_march,rain_march, tmin_april,tmax_april,rain_april

In [20]:
data = []
for district in dist_list_values:
    adf = my[my.distname==district]
    a_shape = adf.geometry

    for i in range(28):
        year = 1995 + i
        values = avg_value(year,a_shape)
        row_data = {'Year': year, 'District':district, 'tmin_march': values[0], 'tmax_march': values[1], 'rain_march': values[2],
                    'tmin_april': values[3], 'tmax_april': values[4], 'rain_april': values[5]}
        data.append(row_data)
        print(i)
df = pd.DataFrame(data)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [21]:
df

,Year,District,tmin_march,tmax_march,rain_march,tmin_april,tmax_april,rain_april
0,1995,Ambedkar Nagar,NaN,NaN,NaN,NaN,NaN,NaN
1,1996,Ambedkar Nagar,NaN,NaN,NaN,NaN,NaN,NaN
2,1997,Ambedkar Nagar,NaN,NaN,NaN,NaN,NaN,NaN
3,1998,Ambedkar Nagar,NaN,NaN,NaN,NaN,NaN,NaN
4,1999,Ambedkar Nagar,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2095,2018,Allahabad,20.802032,36.479344,20.802032,24.534117,39.073861,24.534117
2096,2019,Allahabad,21.231770,36.156461,21.231770,24.562214,40.184507,24.562214
2097,2020,Allahabad,20.505407,34.554676,20.505407,23.242434,38.432839,23.242434
2098,2021,Allahabad,20.174568,36.291604,20.174568,23.780184,38.636488,23.780184


In [22]:
df.to_csv(r"C:\Users\Kanishk Goyal\OneDrive - IIT Kanpur\Desktop\Prof. Hamim Zafar\Data\temp&rain\rain_temp_03_04_UP.csv", index=False)